In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tkinter.filedialog as tkf
import hyperspy.api as hys
import tifffile
from scipy import ndimage
plt.rcParams['font.family'] = 'Times New Roman'
# create a customized colorbar
color_rep = ["black", "orange", "purple", "blue", "red", "green", "yellow", "lime", 
             "cyan", "magenta", "lightgray", "peru", "springgreen", "deepskyblue", 
             "hotpink", "darkgray"]
print(len(color_rep))

In [ ]:
def rif_to_rdf(rif, r_list, k_list):
    gr = []
    profile = rif / np.max(rif)
    for i in range(len(r_list)):
        sin_rk = np.sin(r_list[i]*k_list)
        rif_sin = np.multiply(profile, sin_rk)
        gr_tmp = 8 * np.pi * np.trapz(rif_sin, dx=(k_list[1]-k_list[0]))
        gr.append(gr_tmp)
    
    return np.asarray(gr)

In [ ]:
raw_adr = tkf.askopenfilename()
print(raw_adr)

In [ ]:
# Load a data (DM)
RIF = hys.load(raw_adr)
print(RIF)
RIF_data = RIF.data.copy()
print(RIF_data.shape)
n_dim = len(RIF_data.shape)
print(n_dim)

In [ ]:
scale = []
origin = []
unit = []
size = []

for i in range(n_dim):
    print(RIF.axes_manager[i].scale, RIF.axes_manager[i].offset, RIF.axes_manager[i].units, RIF.axes_manager[i].size)
    scale.append(RIF.axes_manager[i].scale)
    origin.append(RIF.axes_manager[i].offset)
    unit.append(RIF.axes_manager[i].units)
    size.append(RIF.axes_manager[i].size)

In [ ]:
k_list = np.arange(origin[n_dim-1], origin[n_dim-1]+scale[n_dim-1]*size[n_dim-1], scale[n_dim-1])
print(k_list.shape)
print(np.max(k_list))

In [ ]:
r_size = 40
r_scale = 0.125 # Angstrom
r_list = np.arange(r_size) * r_scale
r_unit = unit[-1][-1]
print(r_list.shape)
print(r_scale)
print(r_unit)
#print(r_list)

In [ ]:
# n_dim = 2
if n_dim == 2:
    Gr = []
    for i in range(size[0]):
        tmp_Gr = rif_to_rdf(RIF_data[i], r_list, k_list)
        Gr.append(tmp_Gr)

    Gr = np.asarray(Gr).reshape(RIF_data.shape[0], r_size)
    print(Gr.shape)
    
elif n_dim == 3:
    Gr = []
    for i in range(size[1]):
        for j in range(size[0]):
            tmp_Gr = rif_to_rdf(RIF_data[i, j], r_list, k_list)
            Gr.append(tmp_Gr)

    Gr = np.asarray(Gr).reshape(RIF_data.shape[0], r_size)
    print(Gr.shape)    

In [ ]:
%matplotlib qt

In [ ]:
%matplotlib inline

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5))
for i, line in enumerate(Gr):
    if i != 0:
        ax.plot(r_list, line, c=color_rep[i], label="%d"%(i))

ax.grid()
ax.set_xlabel(unit[-1][-1], fontsize=15)
ax.tick_params(axis="both", labelsize=15)
fig.tight_layout()
#fig.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5))
for i, line in enumerate(Gr):
    ax.plot(r_list, ndimage.gaussian_filter(line, sigma=2), c=color_rep[i], label="%d"%(i))

ax.grid()
ax.set_xlabel(unit[-1][-1], fontsize=15)
ax.tick_params(axis="both", labelsize=15)
fig.tight_layout()
#fig.legend()
plt.show()

In [ ]:
tifffile.imwrite(tkf.asksaveasfilename(), Gr)